In [1]:
pip install fastFM


In [2]:
from fastFM import als,sgd
from sklearn.feature_extraction import DictVectorizer
import gzip
import math
import json
import numpy as np
import random
import pandas as pd
import sklearn
import string
from collections import defaultdict
from nltk.stem.porter import *
from sklearn.linear_model import LogisticRegression,Ridge,Lasso
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,mean_squared_error,confusion_matrix,classification_report,mean_absolute_error,precision_score,recall_score,f1_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from sklearn.feature_selection import SelectKBest,chi2,mutual_info_classif
from sklearn.model_selection import train_test_split
import datetime as dt
#nltk.download('all')
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim.models import Word2Vec
import dateutil
from scipy.sparse import lil_matrix # To build sparse feature matrices, if you like
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from scipy import spatial

In [3]:
train=pd.read_csv('processed_train.csv')
test=pd.read_csv('processed_test.csv')

In [4]:
train['user_id']=train['user_id'].astype('str')
train['item_id']=train['item_id'].astype('str')
train['age']=train['age'].astype('str')
train['size']=train['size'].astype('str')
train['year']=train['year'].astype('str')
train['month']=train['month'].astype('str')
#train['review_polarity_score']=train['review_polarity_score'].apply(lambda x: 1 if (x>0.5) else 0 )
train.drop(['BMI','rented for'],axis=1,inplace=True)
#train.drop(['height','weight',],axis=1,inplace=True)

In [5]:
test['user_id']=test['user_id'].astype('str')
test['item_id']=test['item_id'].astype('str')
test['age']=test['age'].astype('str')
test['size']=test['size'].astype('str')
test['year']=test['year'].astype('str')
test['month']=test['month'].astype('str')
#test['review_polarity_score']=test['review_polarity_score'].apply(lambda x: 1 if (x>0.5) else 0 )
test.drop(['BMI','rented for'],axis=1,inplace=True)
#test.drop(['height','weight','rented for'],axis=1,inplace=True)

In [6]:
train

,fit,user_id,bust size,item_id,weight,rating,body type,category,height,size,age,review_polarity_score,year,month
0,fit,644617,34a,276763,150lbs,8,pear,dress,"5' 6""",8,30,0.9501,2016,6
1,fit,646060,32c,174086,135lbs,10,athletic,dress,"5' 8""",8,26,0.9416,2014,5
2,small,634534,36b,1434272,140lbs,6,pear,sheath,"5' 10""",24,38,0.8021,2016,11
3,large,269132,34b,181737,128lbs,8,straight & narrow,dress,"5' 9""",4,49,0.9273,2014,6
4,large,380701,34c,143094,128lbs,8,pear,dress,"5' 4""",8,30,0.9905,2013,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153957,fit,182654,34dd,1717057,150lbs,8,hourglass,dress,"5' 4""",20,44,0.9442,2016,11
153958,fit,318969,34b,180843,140lbs,10,hourglass,dress,"5' 6""",12,31,0.8267,2012,9
153959,fit,271629,36d,986324,140lbs,10,full bust,dress,"5' 8""",14,29,0.8446,2016,2
153960,fit,761195,38dd,271412,140lbs,8,full bust,sheath,"5' 3""",32,45,0.8479,2017,12


In [7]:
train.dtypes

fit                       object
user_id                   object
bust size                 object
item_id                   object
weight                    object
rating                     int64
body type                 object
category                  object
height                    object
size                      object
age                       object
review_polarity_score    float64
year                      object
month                     object
dtype: object

In [8]:
target_train=train['rating']
features_train=train.drop('rating',axis=1)
train_dict=features_train.to_dict('records')

In [9]:
target_test=test['rating']
features_test=test.drop('rating',axis=1)
test_dict=features_test.to_dict('records')

In [10]:
v = DictVectorizer()
X = v.fit_transform(train_dict)

X_train, X_valid, y_train, y_valid = train_test_split(X, target_train.values, test_size=0.12, random_state=472)

In [11]:
print(X[0])

  (0, 27)	1.0
  (0, 91)	1.0
  (0, 128)	1.0
  (0, 213)	1.0
  (0, 265)	1.0
  (0, 282)	1.0
  (0, 4195)	1.0
  (0, 6068)	1.0
  (0, 6072)	0.9501
  (0, 6127)	1.0
  (0, 60893)	1.0
  (0, 96757)	1.0
  (0, 96902)	1.0


In [12]:
def get_even_rating(val):
  if(val>10):
    return 10
  if (math.floor(val)%2 == 0):
    return int(val)
  else:
    return int(val+1)

In [ ]:
get_even_rating(8.99)

8

In [ ]:
#parameters=[{'n_iter':50, 'rank':8, 'l2_reg_w':0.01, 'l2_reg_V':0.1},{'n_iter':70, 'rank':10, 'l2_reg_w':0.09, 'l2_reg_V':0.2},{'n_iter':80, 'rank':12, 'l2_reg_w':0.1, 'l2_reg_V':0.4},{'n_iter':100, 'rank':15, 'l2_reg_w':0.1, 'l2_reg_V':0.5},{'n_iter':150, 'rank':20, 'l2_reg_w':0.2, 'l2_reg_V':0.7}]

In [ ]:
# Few parameters are known to perform well let us tweak more hyperparameters closely
parameters=[{'n_iter':100, 'rank':15, 'l2_reg_w':0.12, 'l2_reg_V':0.4},{'n_iter':110, 'rank':17, 'l2_reg_w':0.13, 'l2_reg_V':0.3},{'n_iter':120, 'rank':15, 'l2_reg_w':0.15, 'l2_reg_V':0.45},{'n_iter':130, 'rank':17, 'l2_reg_w':0.17, 'l2_reg_V':0.5},{'n_iter':140, 'rank':18, 'l2_reg_w':0.2, 'l2_reg_V':0.3}]

In [ ]:
regression=defaultdict(list)
classification=defaultdict(list)
params=[]

In [ ]:
sparse_test=v.transform(test_dict)

In [ ]:
for d in parameters:
  fm = als.FMRegression(n_iter=d['n_iter'], rank=d['rank'], l2_reg_w=d['l2_reg_w'], l2_reg_V=d['l2_reg_V'])
  #Fit the model to the training data
  fm.fit(X_train, y_train)

  y_pred = fm.predict(X_valid)

  mse_valid = mean_squared_error(y_valid, y_pred)
  mae_valid = mean_absolute_error(y_valid, y_pred)
  rmse_valid = mean_squared_error(y_valid, y_pred,squared=False)

  y_pred_class=[get_even_rating(y) for y in y_pred]
  accuracy_valid=accuracy_score(y_valid,y_pred_class)
  f1_valid=f1_score(y_valid,y_pred_class,average='weighted')

  y_pred_test=fm.predict(sparse_test)

  mse_test = mean_squared_error(target_test.values, y_pred_test)
  mae_test = mean_absolute_error(target_test.values, y_pred_test)
  rmse_test = mean_squared_error(target_test.values, y_pred_test,squared=False)

  y_pred_test_class=[get_even_rating(y) for y in y_pred_test]
  accuracy_test=accuracy_score(target_test.values,y_pred_test_class)
  f1_test=f1_score(target_test.values,y_pred_test_class,average='weighted')

  params.append(d)

  regression['mse_valid'].append(mse_valid)
  regression['rmse_valid'].append(rmse_valid)
  regression['mae_valid'].append(mae_valid)
  regression['mse_test'].append(mse_test)
  regression['rmse_test'].append(rmse_test)
  regression['mae_test'].append(mae_test)

  classification['accuracy_valid'].append(accuracy_valid)
  classification['accuracy_test'].append(accuracy_test)
  classification['f1_valid'].append(f1_valid)
  classification['f1_test'].append(f1_test)

  print(f'For Params {d} we have valid MSE {mse_valid},MAE {mae_valid},RMSE {rmse_valid} and test MSE {mse_test},MAE {mae_test},RMSE {rmse_test}')
  print(f'For Params {d} we have valid accuracy {accuracy_valid}, F1 {f1_valid} and test accuracy {accuracy_test}, F1 {f1_test}')


For Params {'n_iter': 100, 'rank': 15, 'l2_reg_w': 0.12, 'l2_reg_V': 0.4} we have valid MSE 3.2531989890375708,MAE 1.3492057755094469,RMSE 1.8036626594342886 and test MSE 3.224701875532315,MAE 1.3432189258460498,RMSE 1.7957454929728531
For Params {'n_iter': 100, 'rank': 15, 'l2_reg_w': 0.12, 'l2_reg_V': 0.4} we have valid accuracy 0.5414591903009309, F1 0.5495344010858255 and test accuracy 0.5385035074045207, F1 0.5474318837751156
For Params {'n_iter': 110, 'rank': 17, 'l2_reg_w': 0.13, 'l2_reg_V': 0.3} we have valid MSE 3.246769179720235,MAE 1.3482673029687742,RMSE 1.8018793466046041 and test MSE 3.2365427597817944,MAE 1.3477745913322043,RMSE 1.7990393991744023
For Params {'n_iter': 110, 'rank': 17, 'l2_reg_w': 0.13, 'l2_reg_V': 0.3} we have valid accuracy 0.5411885689543191, F1 0.5497327658281522 and test accuracy 0.5366328916601715, F1 0.5453213875587568
For Params {'n_iter': 120, 'rank': 15, 'l2_reg_w': 0.15, 'l2_reg_V': 0.45} we have valid MSE 3.2217397920674635,MAE 1.342708776404

In [ ]:
reg=pd.DataFrame()
reg['Parameters']=params
reg['MSE_validation']=regression['mse_valid']
reg['MAE_validation']=regression['mae_valid']
reg['RMSE_validation']=regression['rmse_valid']
reg['MSE_test']=regression['mse_test']
reg['MAE_test']=regression['mae_test']
reg['RMSE_test']=regression['rmse_test']

In [ ]:
clas=pd.DataFrame()
clas['Parameters']=params
clas['Accuracy_validation']=classification['accuracy_valid']
clas['F1_score_validation']=classification['f1_valid']
clas['Accuracy_test']=classification['accuracy_test']
clas['F1_score_test']=classification['f1_test']

In [ ]:
reg
reg.to_csv('FastFM_regression_mse.csv')

In [ ]:
clas
clas.to_csv('FastFM_classification_accuracy.csv')

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid={'n_iter':np.arange(80,150,15),'rank':np.arange(8,25,2),'l2_reg_w':np.arange(0,0.2,0.03),'l2_reg_V':np.arange(0,1,0.1)}
fm=als.FMRegression()
cv=GridSearchCV(fm,param_grid=param_grid,cv=10,scoring='neg_mean_squared_error') #,'neg_mean_absolute_error'
cv.fit(X_train, y_train) #refit='neg_mean_absolute_error'

In [54]:
#Initialize the model
fm = als.FMRegression(n_iter=16, rank=12, l2_reg_w=0.09, l2_reg_V=0.08)
#Fit the model to the training data
fm.fit(X_train, y_train)

FMRegression(l2_reg_V=0.08, l2_reg_w=0.09, n_iter=16, rank=12)

In [62]:
y_pred = fm.predict(X_valid)

y_pred_class=[get_even_rating(y) for y in y_pred]

mse_valid = mean_squared_error(y_valid, y_pred)
rmse_valid= mean_squared_error(y_valid, y_pred,squared=False)
accuracy_valid=accuracy_score(y_valid,y_pred_class)

print(f'Mean Squared Error on validation set is : {mse_valid}')
print(f'Root Mean Squared Error on validation set is : {rmse_valid}')
print(f'Accuracy on validation set is : {accuracy_valid}')

Mean Squared Error on validation set is : 2.4821819558599767
Root Mean Squared Error on validation set is : 1.5754941941689207
Accuracy on validation set is : 0.564949123186837


In [57]:
mae_valid = mean_absolute_error(y_valid, y_pred)

print(f'Mean Absolute Error on validation set is : {mae_valid}')

Mean Absolute Error on validation set is : 1.170917417121993


In [58]:
sparse_test=v.transform(test_dict)
y_pred_test=fm.predict(sparse_test)
y_pred_test_int=[get_even_rating(y) for y in y_pred_test]

In [59]:
mse_test = mean_squared_error(target_test.values, y_pred_test_int)

print(f'Mean Squared Error on test set is : {mse_test}')

Mean Squared Error on test set is : 2.6605352039490775


In [63]:
rmse_test = mean_squared_error(target_test.values, y_pred_test_int,squared=False)

print(f'Root Mean Squared Error on test set is : {rmse_test}')

Root Mean Squared Error on test set is : 1.6311147120754805


In [60]:
mae_test = mean_absolute_error(target_test.values, y_pred_test_int)

print(f'Mean absolute Error on test set is : {mae_test}')

Mean absolute Error on test set is : 1.0116393868537283


In [61]:
accuracy_test=accuracy_score(target_test.values, y_pred_test_int)
print(f'Accuracy on test set is : {accuracy_test}')

Accuracy on validation set is : 0.5604312808521694


In [ ]:
confusion_matrix(target_test.values, y_pred_test_int)